In [0]:
# Auto Loaderを使用してVolumeからCSVファイルを読み込み
df = (
  spark.readStream                      # ストリーミング読み込みを開始
  .format("cloudFiles")                 # Auto Loaderフォーマットを指定
  .option("cloudFiles.format", "csv")   # ソースファイル形式をCSVに設定
  .option("header", "true")             # ヘッダー行ありと指定
  # CSVファイルのスキーマを事前定義（データ型を明確化）
  .schema("transaction_id INT, customer_id INT, product_id INT, store_id INT, employee_id INT, quantity INT, purchase_date DATE, _rescued_data STRING")
  .load("/Volumes/users/yukiteru_koide/volume_yukiteru_mart_transaction")  # Volumeパスを指定
  .filter("transaction_id IS NOT NULL") # データ品質管理：transaction_idがNULLのレコードを除外
)

# Deltaテーブルに書き込み
df.writeStream.format("delta") \
  .option("checkpointLocation", "/Volumes/users/yukiteru_koide/volume_yukiteru_mart_transaction") \  # チェックポイント保存場所
  .trigger(once=True) \                 # 一度だけ実行（バッチ処理）
  .toTable("users.yukiteru_koide.transactions_table")  # 対象Deltaテーブル
